In [1]:
from aaff import aaff
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
import basis_set_exchange as bse

In [2]:
mol=gto.M(atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))

In [3]:
hfCO=scf.RHF(mol)
hfCO.scf()

converged SCF energy = -112.786616218695


-112.78661621869524

In [106]:
U,dP,e1=alch_deriv(hfCO,[[0,1],[1,-1]])

In [107]:
hfCO.Gradients().grad()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    -0.0000019315
1 O    -0.0000000000    -0.0000000000     0.0000019315
----------------------------------------------


array([[ 1.12940717e-14,  7.03932964e-14, -1.93146126e-06],
       [-1.12940717e-14, -7.03932964e-14,  1.93146268e-06]])

In [108]:
hfCO.Gradients().grad_elec()

array([[ 1.12940717e-14,  7.03932964e-14, -1.10657165e+01],
       [-1.12940717e-14, -7.03932964e-14,  1.10657165e+01]])

In [109]:
from FDcoeffs import *

In [110]:
dl=.5
gs=[]
ge=[]
amds=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_1e(),max_cycle=100)
    g=mf.Gradients()
    gs.append(g.grad())
    ge.append(g.grad_elec())
    amds.append(aaff(mf,[1,-1]))

In [111]:
ge[4]

array([[-6.54751686e-15, -1.59883379e-15, -1.10657172e+01],
       [ 6.54751686e-15,  1.59883379e-15,  1.10657172e+01]])

In [112]:
#forward difference 
(ge[5]-ge[4])/dl

array([[ 9.81206574e-15,  1.69884481e-15, -5.23677703e-01],
       [-9.81206574e-15, -1.69884481e-15,  5.23677703e-01]])

In [113]:
print("2 order finite diff. accuracy:: \n",np.einsum('i,ijk->jk',fd1_2,(np.asarray(ge)))/dl)
print("4 order finite diff. accuracy:: \n",np.einsum('i,ijk->jk',fd1_4,(np.asarray(ge)))/dl)
print("6 order finite diff. accuracy:: \n",np.einsum('i,ijk->jk',fd1_6,(np.asarray(ge)))/dl)
print("8 order finite diff. accuracy:: \n",np.einsum('i,ijk->jk',fd1_8,(np.asarray(ge)))/dl)

2 order finite diff. accuracy:: 
 [[-7.69240108e-14 -4.78306248e-14 -6.77666144e-01]
 [ 7.69240108e-14  4.78306248e-14  6.77666144e-01]]
4 order finite diff. accuracy:: 
 [[-9.65355689e-14 -5.87376648e-14 -6.85183479e-01]
 [ 9.65355689e-14  5.87376648e-14  6.85183479e-01]]
6 order finite diff. accuracy:: 
 [[-1.12443086e-13 -5.84824917e-14 -6.85575233e-01]
 [ 1.12443086e-13  5.84824917e-14  6.85575233e-01]]
8 order finite diff. accuracy:: 
 [[-1.26475518e-13 -5.44491430e-14 -6.85618026e-01]
 [ 1.26475518e-13  5.44491430e-14  6.85618026e-01]]


In [114]:
aaff(hfCO,[1,-1])

array([[-4.84788999e-14, -1.58726207e-13, -6.85627204e-01],
       [ 4.84788999e-14,  1.58726207e-13,  6.85627204e-01]])

In [115]:
print(np.einsum('i,ijk->jk',fd2_8,(np.asarray(ge)))/dl**2)
print(np.einsum('i,ijk->jk',fd1_8,(np.asarray(amds)))/dl)

[[ 5.38800761e-13  2.67006580e-13  6.19053615e-01]
 [-5.38800761e-13 -2.67006580e-13 -6.19053615e-01]]
[[-3.53437475e-13 -3.64653912e-13  6.18863830e-01]
 [ 3.53437475e-13  3.64653912e-13 -6.18863830e-01]]


In [116]:
print(np.einsum('i,ijk->jk',fd3_6,(np.asarray(ge)))/dl**3)
print(np.einsum('i,ijk->jk',fd2_6,(np.asarray(amds)))/dl**2)

[[ 1.40629569e-12  1.22357710e-13  1.93566569e-01]
 [-1.40629569e-12 -1.22357710e-13 -1.93566569e-01]]
[[ 1.53441311e-12  1.28069829e-12  1.93908091e-01]
 [-1.53441311e-12 -1.28069829e-12 -1.93908091e-01]]


In [117]:
print(np.einsum('i,ijk->jk',fd4_2,(np.asarray(ge)))/dl**4)
print(np.einsum('i,ijk->jk',fd3_6,(np.asarray(amds)))/dl**3)

[[-4.79491070e-12 -2.53470013e-12 -1.36370530e-01]
 [ 4.79491070e-12  2.53470013e-12  1.36370530e-01]]
[[ 1.92570842e-12  4.23547065e-12 -1.47621861e-01]
 [-1.92570842e-12 -4.23547065e-12  1.47621861e-01]]


In [118]:
print(np.einsum('i,ijk->jk',fd6_4,(np.asarray(ge)))/dl**6)
print(np.einsum('i,ijk->jk',fd5_4,(np.asarray(amds)))/dl**5)
#print(np.einsum('i,ijk->jk',fd6_4,(np.asarray(amds)))/dl**6)

[[ 1.62199506e-10  2.43917894e-11  4.21289969e-01]
 [-1.62199506e-10 -2.43917894e-11 -4.21289969e-01]]
[[ 6.43215287e-14 -4.59198987e-11  3.43309838e-01]
 [-6.43215287e-14  4.59198987e-11 -3.43309838e-01]]


In [119]:
#with dl=.1  the 4th derivatives are:   only McL expansion of aaff is consistent to obtain APDFT4_g1 derivs
#[[-5.43974711e-09 -4.46784523e-09 -8.97038551e-02]
# [ 5.43974711e-09  4.46784523e-09  8.97038538e-02]]
#[[ 1.04025962e-09  1.59691820e-09 -1.55567761e-01]
# [-2.70775954e-08  7.58253933e-09  1.56759171e-01]]

In [120]:
NN=FcM(fcs=[1,-1],atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
hfNN=scf.RHF(NN)
hfNN.scf(dm0=hfNN.init_guess_by_1e())
gNN=hfNN.Gradients().grad_elec()

converged SCF energy = -108.984680830552


In [121]:
gNN,hfNN.Gradients().grad_nuc()+gNN

(array([[-4.82991281e-16, -6.88769736e-16, -1.14171161e+01],
        [ 4.82991281e-16,  6.88769736e-16,  1.14171161e+01]]),
 array([[-4.82991281e-16, -6.88769736e-16, -1.20865800e-01],
        [ 4.82991281e-16,  6.88769736e-16,  1.20865800e-01]]))

In [122]:
hfNN.Gradients().grad_nuc()+ge[4]+amds[4]+np.einsum('i,ijk->jk',fd1_2,(np.asarray(amds)))/dl/2  \
    +np.einsum('i,ijk->jk',fd2_2,(np.asarray(amds)))/dl**2/6+np.einsum('i,ijk->jk',fd3_6,(np.asarray(amds)))/dl**3/24 \
  +np.einsum('i,ijk->jk',fd4_4,(np.asarray(amds)))/dl**4/120+np.einsum('i,ijk->jk',fd5_4,(np.asarray(amds)))/dl**5/720 \
 +np.einsum('i,ijk->jk',fd6_2,(np.asarray(amds)))/dl**6/5040

array([[-4.17944934e-14, -8.61170009e-15, -1.24588144e-01],
       [ 4.17944934e-14,  8.61170009e-15,  1.24588144e-01]])

In [128]:
hfNN.Gradients().grad_nuc()+ge[4]+amds[4]/2,amds[4]

(array([[-4.40412411e-15, -4.47724860e-15, -1.12280522e-01],
        [ 4.40412411e-15,  4.47724860e-15,  1.12280522e-01]]),
 array([[ 4.28678551e-15, -5.75682961e-15, -6.85627119e-01],
        [-4.28678551e-15,  5.75682961e-15,  6.85627119e-01]]))

In [130]:
(hfNN.Gradients().grad_nuc()-hfCO.Gradients().grad_nuc())/(-2)

array([[-0.        , -0.        , -0.11526786],
       [-0.        , -0.        ,  0.11526786]])

In [124]:
aaff_predict=[]
aaff_predict.append((hfNN.Gradients().grad_nuc()+ge[4])[1,2])
aaff_predict.append(aaff_predict[0]+amds[4][1,2])
aaff_predict.append(aaff_predict[1]+np.einsum('i,ijk->jk',fd1_8,(np.asarray(amds)))[1,2]/dl/2)
aaff_predict.append(aaff_predict[2]+np.einsum('i,ijk->jk',fd2_6,(np.asarray(amds)))[1,2]/dl**2/6)
aaff_predict.append(aaff_predict[3]+np.einsum('i,ijk->jk',fd3_6,(np.asarray(amds)))[1,2]/dl**3/24 )
aaff_predict.append(aaff_predict[4]+np.einsum('i,ijk->jk',fd4_4,(np.asarray(amds)))[1,2]/dl**4/120)
aaff_predict.append(aaff_predict[5]+np.einsum('i,ijk->jk',fd5_4,(np.asarray(amds)))[1,2]/dl**5/720)


In [125]:
aaff_predict

[-0.23053303698926975,
 0.45509408197926005,
 0.14566216699861612,
 0.1133441518937541,
 0.11949506276026936,
 0.12139314501284991,
 0.12091632579319032]

In [126]:
fd_predict=[]
fd_predict.append((hfNN.Gradients().grad_nuc()+ge[4])[1,2])
fd_predict.append(fd_predict[0]+np.einsum('i,ijk->jk',fd1_8,(np.asarray(ge)))[1,2]/dl)
fd_predict.append(fd_predict[1]+np.einsum('i,ijk->jk',fd2_6,(np.asarray(ge)))[1,2]/dl**2/2)
fd_predict.append(fd_predict[2]+np.einsum('i,ijk->jk',fd3_6,(np.asarray(ge)))[1,2]/dl**3/6)
fd_predict.append(fd_predict[3]+np.einsum('i,ijk->jk',fd4_4,(np.asarray(ge)))[1,2]/dl**4/24 )
fd_predict.append(fd_predict[4]+np.einsum('i,ijk->jk',fd5_4,(np.asarray(ge)))[1,2]/dl**5/120)
fd_predict.append(fd_predict[5]+np.einsum('i,ijk->jk',fd6_2,(np.asarray(ge)))[1,2]/dl**6/720)
fd_predict

[-0.23053303698926975,
 0.4550849889243995,
 0.14558856794563008,
 0.11332747303117582,
 0.11950464531185602,
 0.12133793045596335,
 0.12094187695199507]

In [ ]:
# trget 1.20865800e-01